In [ ]:
from wildfires.logging_config import enable_logging
import logging
from wildfires.analysis.analysis import *
from wildfires.analysis.plotting import *
from wildfires.data.datasets import *
from wildfires.data.cube_aggregation import *
from wildfires.utils import *
from IPython.display import HTML
import numpy as np
import matplotlib as mpl
from matplotlib import animation

import os

FigureSaver.directory = os.path.expanduser(os.path.join("~", "tmp", "animation"))
os.makedirs(FigureSaver.directory, exist_ok=True)
logger = logging.getLogger(__name__)

enable_logging("jupyter")
warnings.filterwarnings("ignore", ".*Collapsing a non-contiguous coordinate.*")
warnings.filterwarnings("ignore", ".*DEFAULT_SPHERICAL_EARTH_RADIUS*")
warnings.filterwarnings("ignore", ".*guessing contiguous bounds*")

normal_coast_linewidth = 0.5
mpl.rc("figure", figsize=(14, 6))
mpl.rc("font", size=9.0)

np.random.seed(1)

In [ ]:
monthly = prepare_selection(Datasets(GFEDv4()).select_variables("GFED4 BA", inplace=True), which="monthly")
monthly.apply_masks(~get_land_mask())

In [ ]:
vmin, vmax = get_cubes_vmin_vmax(monthly.cubes)
n_lat = len(monthly.cube.coord("latitude").points)
n_lon = len(monthly.cube.coord("longitude").points)

mpl.rcParams['animation.embed_limit'] = 500.0

# N_frames = monthly.cube.shape[0]
N_frames = 10
interval = 1000 / 12 # One second per year.

mpl.rc("figure", figsize=(22, 14))

fig, ax, mesh, suptitle_text = cube_plotting(
    monthly.cube[0],
log=True,
animation_output=True,
title='',
vmin=vmin,
vmax=vmax,
    cmap="Reds",
    cbar_tick_size=15,
    cbar_label_size=14,
    orientation="horizontal",
    shrink=0.65,
    fraction=0.05,
    aspect=40,
)

title_text = ax.text(
0.5, 1.08, "placeholder", transform=ax.transAxes, ha="center", fontsize=24)
plt.close()

def init():
    mesh.set_array(np.zeros(n_lat * n_lon))
    title_text.set_text("")
    return (mesh,)

with TqdmContext(unit="plot", desc="Plotting", total=N_frames) as tqdm_context:
    def animate(i):        
        single_time_cube = monthly.cube[i]
        _ = cube_plotting(
        single_time_cube,
        log=True,
        ax=ax,
        mesh=mesh,
        animation_output=False,
        new_colorbar=False,
        title="",
        )
        title_text.set_text(
            # Ignore the time, which flip-flops between 00:00:00 and 12:00:00.
            f"{monthly.dataset.pretty} "
            + str(single_time_cube.coord("time").cell(0).point)[:7]
        )
        tqdm_context.update()        

        return mesh, title_text

    anim = animation.FuncAnimation(
        fig, animate, init_func=init, frames=N_frames, interval=interval, blit=True)

    # out = anim.to_jshtml()
# HTML(out)
anim.save('test.html', writer=animation.writers['html'](fps=10))